In [4]:
import pandas as pd
from pyvi import ViTokenizer
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

# Khởi tạo scorer cho ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

# Hàm tách từ tiếng Việt
def tokenize_vi(text):
    return ViTokenizer.tokenize(text)

def compute_meteor_vi(ref, pred):
    ref_tok = tokenize_vi(ref).split()
    pred_tok = tokenize_vi(pred).split()
    return meteor_score([ref_tok], pred_tok)
# Tính ROUGE cho tiếng Việt
def compute_rouge_vi(ref, pred):
    ref_tok = tokenize_vi(ref)
    pred_tok = tokenize_vi(pred)
    scores = scorer.score(ref_tok, pred_tok)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure,
    }

# Đọc dữ liệu
# df = pd.read_csv('your_file.csv')

# Giả sử bạn đã có df với cột: 'reference' và 'response'



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [50]:
df = pd.read_csv("C:/Users/user/OneDrive/Desktop/LLMs for Edu/RAFT_Data/openai_eval/res_after_eval/llama_finetuned_CoT.csv")

In [51]:
df = df.dropna(subset=['reference', 'response'])
df['reference'] = df['reference'].astype(str)
df['response'] = df['response'].astype(str)


In [52]:
df['meteor'] = df.apply(lambda row: compute_meteor_vi(row['reference'], row['response']), axis=1)
rouge_scores = df.apply(lambda row: compute_rouge_vi(row['reference'], row['response']), axis=1)
rouge_df = pd.DataFrame(rouge_scores.tolist())

# Gộp kết quả
df = pd.concat([df, rouge_df], axis=1)



In [53]:
rouge_df.mean()

rouge1    0.668188
rouge2    0.523086
rougeL    0.549897
dtype: float64

In [54]:
mean_values = df[["meteor"]].mean()
mean_values

meteor    0.529846
dtype: float64